In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                    'key2' : ['one', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.289462,1.291418
1,a,two,-0.445753,-0.425030
2,b,one,0.607605,0.691535
3,b,two,-1.125362,-0.286486
4,a,one,-0.396070,0.536314


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.377095
b   -0.258878
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.342766
      two    -0.445753
b     one     0.607605
      two    -1.125362
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.342766,-0.445753
b,0.607605,-1.125362


In [8]:
# the group keys could be any arrays of the right length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df["data1"].groupby([states, years]).mean()

California  2005   -0.445753
            2006    0.607605
Ohio        2005   -0.707412
            2006   -0.396070
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.377095,0.467567
b,-0.258878,0.202524


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.342766  0.913866
     two  -0.445753 -0.425030
b    one   0.607605  0.691535
     two  -1.125362 -0.286486

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.289462  1.291418
1    a  two -0.445753 -0.425030
4    a  one -0.396070  0.536314
b
  key1 key2     data1     data2
2    b  one  0.607605  0.691535
3    b  two -1.125362 -0.286486


In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.289462  1.291418
4    a  one -0.396070  0.536314
('a', 'two')
  key1 key2     data1    data2
1    a  two -0.445753 -0.42503
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.607605  0.691535
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.125362 -0.286486


In [14]:
pieces = dict(list(df.groupby('key1')))
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.289462,1.291418
1,a,two,-0.445753,-0.425030
4,a,one,-0.396070,0.536314


In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [16]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.289462  1.291418
1 -0.445753 -0.425030
2  0.607605  0.691535
3 -1.125362 -0.286486
4 -0.396070  0.536314
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [17]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [18]:
# same as below
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [19]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.913866
     two  -0.425030
b    one   0.691535
     two  -0.286486

In [20]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [21]:
s_grouped.mean()

key1  key2
a     one     0.913866
      two    -0.425030
b     one     0.691535
      two    -0.286486
Name: data2, dtype: float64

## Grouping with Dicts and Series

In [22]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [23]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-1.643859,-0.879534,1.440920,1.298140,0.247892
Steve,2.018499,1.185848,0.347716,-0.138036,0.387433
Wes,-0.795530,NaN,NaN,0.855571,-0.599762
Jim,-0.823949,-1.704101,0.010641,-1.257923,-1.257200
Travis,0.862242,0.639823,-1.840470,-0.237137,-0.933502


In [24]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
            'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,2.739060,-2.275501
Steve,0.209679,3.591780
Wes,0.855571,-1.395292
Jim,-1.247283,-3.785250
Travis,-2.077606,0.568562


In [25]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [26]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with Functions

In [27]:
people

,a,b,c,d,e
Joe,-1.643859,-0.879534,1.440920,1.298140,0.247892
Steve,2.018499,1.185848,0.347716,-0.138036,0.387433
Wes,-0.795530,NaN,NaN,0.855571,-0.599762
Jim,-0.823949,-1.704101,0.010641,-1.257923,-1.257200
Travis,0.862242,0.639823,-1.840470,-0.237137,-0.933502


In [28]:
# compute an array of string lengths
people.groupby(len).sum()

,a,b,c,d,e
3,-3.263338,-2.583635,1.451561,0.895787,-1.609070
5,2.018499,1.185848,0.347716,-0.138036,0.387433
6,0.862242,0.639823,-1.840470,-0.237137,-0.933502


In [29]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.643859 -0.879534  1.440920  0.855571 -0.599762
  two -0.823949 -1.704101  0.010641 -1.257923 -1.257200
5 one  2.018499  1.185848  0.347716 -0.138036  0.387433
6 two  0.862242  0.639823 -1.840470 -0.237137 -0.933502

## Grouping by Index Levels

In [30]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

In [31]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.483110 -0.184255 -0.738450 -0.814307 -1.183866
1      0.604809  0.782164  0.750350  0.053374  0.497655
2      2.168204  1.373414  1.432321  1.291181 -2.152649
3      0.957114 -0.552456 -0.690294 -0.006209 -0.542119

In [32]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
